In [1]:
from time import *
from ephem import *
import math
import datetime
#
today=strftime("%Y/%m/%d")
horizon='20:00' # can also give in radians,

KAT=Observer()
KAT.long,KAT.lat='21:24:38.46','-30:43:17.34' # positive East,North
#KAT.date=today
#KAT.epoch=today
KAT.date='2018/01/05'
KAT.epoch=KAT.date
# print KAT.epoch
KAT.elev=1038            #-metres above MSL
KAT.temp=14             # temperature in C
KAT.pressure=0         # pressure in mbar set to zero to ignore refraction
KAT.horizon=horizon    # minimum elevation
dd=KAT.date.tuple()
print "Date %d/%d/%d UT %02d:%02d  LST %s Horizon %s"\
      %(dd[0],dd[1],dd[2],dd[3],dd[4],KAT.sidereal_time(),str(KAT.horizon))
print "Source\t\t  RA \t\t Dec \t  UTRise    UTTransit UTSet"
#
sun=Sun()
sun.compute(KAT)
#print "Sun  Alt",  sun.alt, "Az", sun.az
print "Sun \t\t%-12s %12s %-9s %-9s %-9s " \
      %(sun.ra,sun.dec,str(KAT.next_rising(sun))[-8:],\
        str(KAT.next_transit(sun))[-8:],str(KAT.next_setting(sun))[-8:])
#
moon=Moon()
moon.compute(KAT)
#print "Moon Alt", moon.alt,"Az", moon.az

print "Moon \t\t%-12s %12s %-9s %-9s %-9s " \
            % (moon.ra,moon.dec,str(KAT.next_rising(moon))[-8:],\
               str(KAT.next_transit(moon))[-8:],str(KAT.next_setting(moon))[-8:])




Date 2018/1/5 UT 00:00  LST 8:23:47.93 Horizon 20:00:00.0
Source		  RA 		 Dec 	  UTRise    UTTransit UTSet
Sun 		19:03:19.73   -22:38:06.5 05:20:37  10:39:43  15:58:43  
Moon 		9:51:59.69     14:42:02.2 22:22:34  01:30:15  05:22:29  


In [2]:
def SAST(utstring): #YYYY/MM/DD hh:mm:ss format
    ut=datetime.datetime.strptime(utstring,"%Y/%m/%d %H:%M:%S")
    sast=ut+datetime.timedelta(hours=2)
    sasts= sast.strftime("%Y/%m/%d %H:%M:%S") # with seconds
    return sasts
    

In [3]:
def UT(day,time):# format YYYY/MM/DD and hh:mm - ignore seconds
    sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime format
    ut=sast- datetime.timedelta(hours=2)
    UTS=ut.strftime("%Y/%m/%d %H:%M:%S") # neat UT string with seconds
    return UTS


In [4]:
def LST(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    Set the time string to Hours:Minutes:Seconds
    """
    KAT.date=ut
    lst=str(KAT.sidereal_time())[:8]
    return lst


In [5]:
def JD(ut):
    """ 
    Requires date and time in YYYY/MM/DD hh:mm:ss format
    returns jd
    """
    jd=julian_date(ut)
    return jd

In [6]:
local="13:45"+":00"
print local
datetime.time()


13:45:00


datetime.time(0, 0)

In [7]:
day="2018/01/05"   #string
time="14:00"       #string
sast=datetime.datetime.strptime(day+time,"%Y/%m/%d%H:%M") # datetime
print sast
ut=sast- datetime.timedelta(hours=2)
print ut
print "SAST",sast.strftime("%Y/%m/%d %H:%M:%S")," UT",ut.strftime("%Y/%m/%d %H:%M:%S")
                                


2018-01-05 14:00:00
2018-01-05 12:00:00
SAST 2018/01/05 14:00:00  UT 2018/01/05 12:00:00


In [8]:
print SAST("2018/01/05 14:00:00")

2018/01/05 16:00:00


In [9]:
x=UT("2018/01/05","14:00")
print x

2018/01/05 12:00:00


In [26]:
KAT.date="2018/01/05 14:00:00"   # UT string
str(KAT.sidereal_time())


'22:26:05.92'

In [11]:
date="2018/01/05"
time="14:00"
ut_test=UT(date,time)
print ut_test
lst=LST(ut_test)
print SAST(ut_test), ut_test, lst

2018/01/05 12:00:00
2018/01/05 14:00:00 2018/01/05 12:00:00 20:25:46


In [12]:
inp=("dec30, J2000,7:00:00,30:00:00", "dec20, J2000, 7:00:00,20:00:00", "dec0, J2000, 7:00:00,00:00:00"\
     , "dec-20, J2000,7:00:00,-20:00:00", "dec-40, J2000,7:00:00,-40:00:00", "dec-60, J2000,7:00:00,-60:00:00")
for line in inp:
  if line[0] != "#":
    s=line.split(",") # split the line into strings
    source=s[0]
# next item is 'radec...'    
    ra=s[2]
    dec=s[3]
    dbline='"'+source+",f|J|Q,"+ra+","+dec+",,2000"+'"'
#    print dbline
    p=readdb(dbline)
#    p=FixedBody()
#    p._ra,p._dec=ra,dec
#    p._epoch='2000'
    p.compute(KAT)
    if p.neverup == True:
       print "%-11s  %12s %12s           Never Rises           " %(source,p.ra,p.dec)
    if p.circumpolar == True:
       transit=str(KAT.next_transit(p))[-8:]
       print "%-11s  %12s %12s          %9s  Circumpolar "  %(source,p.ra,p.dec,transit)
    if p.neverup == False and p.circumpolar == False:  # Source rises and sets
#      print source, p.ra, p.dec, p.alt, p.az,
      rises=str(KAT.next_rising(p))[-8:]
      transit=str(KAT.next_transit(p))[-8:]
      sets=str(KAT.next_setting(p))[-8:]
      uptime=((KAT.next_setting(p)-KAT.next_rising(p))%1)*24  # 
#calculation is in days, so make modulo 1 day, times 24 for hours.
      print "%-12s %12s %12s %-9s %-9s %-9s  %5.2f " % (source,p.ra,p.dec,rises,transit,sets,uptime)
#    for n in range(3,len(s)):    #comments at the end of the input line
#         print s[n],
#    print ""


dec30          7:01:09.46   29:58:17.9 20:07:20  22:33:39  00:59:58    4.88 
dec20          7:01:04.46   19:58:17.8 19:13:04  22:33:34  01:54:04    6.68 
dec0           7:00:56.07   -0:01:42.0 18:07:52  22:33:26  02:59:00    8.85 
dec-20         7:00:47.85  -20:01:41.7 17:21:54  22:33:18  03:44:42   10.38 
dec-40         7:00:37.32  -40:01:41.0 16:38:46  22:33:07  04:27:28   11.81 
dec-60         7:00:17.86  -60:01:40.1 15:39:47  22:32:48  05:25:49   13.77 


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: the ephem.Body attributes 'rise_time', 'rise_az', 'transit_time', 'transit_alt', 'set_time', 'set_az', 'circumpolar', and 'never_up' are deprecated; please convert your program to use the ephem.Observer functions next_rising(), previous_rising(), next_transit(), and so forth

  app.launch_new_instance()


In [13]:
print p.alt,p.az

20:00:00.0 148:52:38.3


In [14]:
KAT

<ephem.Observer date='2018/1/5 12:00:00' epoch='2018/1/5 00:00:00' lon='21:24:38.5' lat='-30:43:17.3' elevation=1038.0m horizon=20:00:00.0 temp=14.0C pressure=0.0mBar>

In [15]:
ut=today+" 14:00:20"
print LST(ut)
print JD(ut)

22:26:25
2458124.08356


In [16]:
KAT.date=now()
print LST(KAT.date)

18:56:58


In [17]:
str(localtime(now()))

'2018-01-05 12:31:26.000004'

In [18]:
now()

43103.93850694444

In [19]:
JD(ut)-now()-JD("1899/12/31 12:00:00")

0.14505787007510662

In [20]:
JD("2015/05/06 12:00:00")

2457149.0

In [21]:
str(now()) # UT time now

'2018/1/5 10:31:27'

In [22]:
def tomorrow():
    tod=datetime.datetime.today().date() #today
    tom=tod+datetime.timedelta(1) # tomorrow
    toms=tom.strftime("%Y/%m/%d")
    return toms

In [23]:
KAT.horizon='00:00' # NOT 20deg like regular sources
sun.compute(KAT)
sunrise=KAT.next_rising(sun)
sunset=KAT.next_setting(sun)
print "Sunset ",localtime(sunset)
print "Sunrise",localtime(sunrise)
#

Sunset  2018-01-05 19:38:24.000004
Sunrise 2018-01-06 05:41:37.000005


In [24]:
print tomorrow()
tom=tomorrow()
print SAST(str(KAT.next_rising(sun,start=tomorrow())))

2018/01/06
2018/01/06 05:41:38


In [25]:

print "LST at sunset ", LST(str(sunset))
print "LST at sunrise", LST(str(KAT.next_rising(sun,start=tom)))
print "SAST Sunset   ", SAST(str(sunset))
print "SAST Sunrise  ", SAST(str(KAT.next_rising(sun,start=tom)))

LST at sunset  2:05:05.
LST at sunrise 12:09:58
SAST Sunset    2018/01/05 19:38:24
SAST Sunrise   2018/01/06 05:41:38
